### Homework 2
Name: Tamanna Urmi, 
Date: 15 March, 2022

#### Problem 2.1: Log-binning

In [ ]:
dt = (np.random.random(10000)*1000)**3
dt.size

In [ ]:
def f(x):
    f = x**-2
    return f


In [ ]:
print(min(data))
print(max(data))

In [ ]:
def logbin_fx(data):
    b0 = np.log10(min(data))
    bn = np.log10(max(data))
    
    # create bins from data
    bin_edges = np.logspace(b0, bn, num=21, endpoint=True, base=10.0, dtype=None, axis=0)
    data_df = pd.DataFrame(data, columns = ['x'])
    data_df['y'] = f(data_df['x'])
    
    # logbin the data
    log_binned_dt = pd.cut(data, bin_edges, right=False)
    data_df['bin'] = log_binned_dt
    
    # calculation various X and the Y
    log_binned_xy = data_df.groupby(['bin']).agg({'x': ["min", "max", "count", "mean"], 'y': ["mean"]}).droplevel(axis=1, level=0).reset_index()
    log_binned_xy.columns = ['bin', 'X_min', 'X_max', 'X_cnt', 'X_mean', 'Y_mean']
    log_binned_xy['X_sqrt_prod'] = log_binned_xy.apply(lambda row: np.sqrt((row['bin'].right)*(row['bin'].left)), axis = 1)
    log_binned_xy['Y_prob'] = log_binned_xy.apply(lambda row: row['X_cnt']/(len(data)*(row['bin'].right - row['bin'].left)), axis = 1)

    return log_binned_xy
    


In [ ]:
log_df = logbin_fx(dt)
log_df

In [ ]:
plt.loglog(log_df.X_min, log_df.Y_mean)

#### Problem 2.2: Degree-dependent quantities.

In [ ]:
n = 10**3
m = int(comb(10**3, 2)*0.7)
seed = 20160

G = nx.gnm_random_graph(n, m, seed=seed)

nx.draw_spring(G, node_size=10)

In [ ]:
# Create degree, triangle count
degrees = {node:val for (node, val) in G.degree()}
triangles = nx.triangles(G)

# Create degree-dependent clustering values
def locclust(row):
    val = row['tri']/comb(row['degree'], 2)
    return val

# Create knn values
knn_val = {}
for main_node in G.nodes():
    knn = 0
    for neigh in G.neighbors(main_node):
        knn += degrees[neigh]
    knn_val[main_node] = knn/degrees[main_node]


# Create dataframe with 4 values
dt_df = pd.DataFrame.from_dict(degrees, orient = 'index', columns = ['degree'])
dt_df['tri'] = triangles.values()
dt_df['loc_clust'] = dt_df.apply(locclust, axis = 1)
dt_df['knn_val'] = knn_val.values()
dt_df = dt_df.sort_values(by=['degree'])

dt_df

In [ ]:

dt_agg_df = dt_df[['degree', 'loc_clust', 'knn_val']].groupby(['degree']).agg({'loc_clust':['mean'], 'knn_val':['mean']}).droplevel(axis=1, level=0).reset_index()
dt_agg_df.columns = ['degree', 'c(k)', 'knn(k)']
dt_agg_df


In [ ]:
def logbin_fx2(df, x_var, y_var):
    data = df[x_var].values
    b0 = np.log10(min(data))
    bn = np.log10(max(data))
    
    # create bins from data
    bin_edges = np.logspace(b0, bn, num=21, endpoint=True, base=10.0, dtype=None, axis=0)
#     display(data)
    data_df = pd.DataFrame(data, columns = ['x'])
    data_df['y'] = df[y_var]
    
    # logbin the data
    log_binned_dt = pd.cut(data, bin_edges, right=False)
    data_df['bin'] = log_binned_dt
#     display(data_df)
    
    # calculation various X and the Y
    log_binned_xy = data_df.groupby(['bin']).agg({'x': ["min", "max", "count", "mean"], 'y': ["mean"]}).droplevel(axis=1, level=0).reset_index()
    log_binned_xy.columns = ['bin', 'X_min', 'X_max', 'X_cnt', 'X_mean', 'Y_mean']
    log_binned_xy['X_sqrt_prod'] = log_binned_xy.apply(lambda row: np.sqrt((row['bin'].right)*(row['bin'].left)), axis = 1)
    log_binned_xy['Y_prob'] = log_binned_xy.apply(lambda row: row['X_cnt']/(len(data)*(row['bin'].right - row['bin'].left)), axis = 1)

    return log_binned_xy

def logbin_px(df, x_var):
    data = df[x_var].values
    b0 = np.log10(min(data))
    bn = np.log10(max(data))
    
    # create bins from data
    bin_edges = np.logspace(b0, bn, num=21, endpoint=True, base=10.0, dtype=None, axis=0)
    data_df = pd.DataFrame(data, columns = ['x'])
    
    # logbin the data
    log_binned_dt = pd.cut(data, bin_edges, right=False)
    data_df['bin'] = log_binned_dt
    
    # calculation various X and the Y
    log_binned_xy = data_df.groupby(['bin']).agg({'x': ["min", "max", "count", "mean"]}).droplevel(axis=1, level=0).reset_index()
    log_binned_xy.columns = ['bin', 'X_min', 'X_max', 'X_cnt', 'X_mean']
    log_binned_xy['X_sqrt_prod'] = log_binned_xy.apply(lambda row: np.sqrt((row['bin'].right)*(row['bin'].left)), axis = 1)
    log_binned_xy['Y_prob'] = log_binned_xy.apply(lambda row: row['X_cnt']/(len(data)*(row['bin'].right - row['bin'].left)), axis = 1)

    return log_binned_xy

In [ ]:
lb_deg = logbin_px(dt_agg_df, 'degree')
lb_deg.head(n=3)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_deg['X_min'], lb_deg['Y_prob'])
ax.set_xlabel('degree (log bin)')
ax.set_ylabel('probability distribution (mean log bin)')


In [ ]:
lb_knn = logbin_fx2(dt_agg_df, 'degree', 'knn(k)')
lb_clust = logbin_fx2(dt_agg_df, 'degree', 'c(k)')

In [ ]:
lb_knn.head(n=3)

In [ ]:
lb_clust.head(n=3)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_knn['X_min'], lb_knn['Y_mean'])
ax.set_xlabel('degree (log bin)')
ax.set_ylabel('knn (mean log bin)')

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_clust['X_min'], lb_clust['Y_mean'])
ax.set_xlabel('degree (log bin)')
ax.set_ylabel('degree dependent clustering (mean log bin)')

#### Problem 2.3: Simplest model of scale-free networks


In [ ]:
## Initialize the graph
n = 10**3
n_list = list(range(n))
G = nx.Graph()
G.add_edge(1, 2)

new_node = 3
while new_node <= n:
    eg = list(G.edges())
    random_e = random.choice(eg)
#     print(random_e)
    G.add_edge(random_e[0], new_node)
    G.add_edge(random_e[1], new_node)
    new_node += 1
    
nx.draw_spring(G, node_size=10)

#### Problem 2.4: Measurements of degree-dependent quantities

In [ ]:
## Initialize the graph
n = 10**5

G2 = nx.Graph()
eg = [(1,2)]
G2.add_edge(1, 2)

new_node = 3
while new_node <= n:
    random_e = random.choice(eg)
    eg.append((random_e[0], new_node))
    eg.append((random_e[1], new_node))
    new_node += 1
    
G2.add_edges_from(eg)
    

In [ ]:
def logbin_px(df, x_var):
    data = df[x_var].values
    b0 = np.log10(min(data))
    bn = np.log10(max(data))
    
    # create bins from data
    bin_edges = np.logspace(b0, bn, num=21, endpoint=True, base=10.0, dtype=None, axis=0)
    data_df = pd.DataFrame(data, columns = ['x'])
    
    # logbin the data
    log_binned_dt = pd.cut(data, bin_edges, right=False)
    data_df['bin'] = log_binned_dt
    
    # calculation various X and the Y
    log_binned_xy = data_df.groupby(['bin']).agg({'x': ["min", "max", "count", "mean"]}).droplevel(axis=1, level=0).reset_index()
    log_binned_xy.columns = ['bin', 'X_min', 'X_max', 'X_cnt', 'X_mean']
    log_binned_xy['X_sqrt_prod'] = log_binned_xy.apply(lambda row: np.sqrt((row['bin'].right)*(row['bin'].left)), axis = 1)
    log_binned_xy['Y_prob'] = log_binned_xy.apply(lambda row: row['X_cnt']/(len(data)*(row['bin'].right - row['bin'].left)), axis = 1)

    return log_binned_xy

In [ ]:
# Create degree, triangle count
degrees = {node:val for (node, val) in G2.degree()}
triangles = nx.triangles(G2)

def locclust(row):
    val = row['tri']/comb(row['degree'], 2)
    return val

# Create knn values
knn_val2 = {}
for main_node in G2.nodes():
    knn = 0
    for neigh in G2.neighbors(main_node):
        knn += degrees[neigh]
    knn_val2[main_node] = knn/degrees[main_node]

# Create dataframe with 4 values
dt_df2 = pd.DataFrame.from_dict(degrees, orient = 'index', columns = ['degree'])
dt_df2['tri'] = triangles.values()
dt_df2['loc_clust'] = dt_df2.apply(locclust, axis = 1)
dt_df2['knn_val'] = knn_val2.values()
dt_df2 = dt_df2.sort_values(by=['degree'])

dt_df2

In [ ]:
dt_agg_df2 = dt_df2[['degree', 'loc_clust', 'knn_val']].groupby(['degree']).agg({'loc_clust':['mean'], 'knn_val':['mean']}).droplevel(axis=1, level=0).reset_index()
dt_agg_df2.columns = ['degree', 'c(k)', 'knn(k)']
dt_agg_df2

In [ ]:
lb_deg2 = logbin_px(dt_df2, 'degree')
lb_deg2

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_deg2['X_min'], lb_deg2['Y_prob'])
ax.set_xlabel('degree, $k$ [log binned]')
ax.set_ylabel('$p(k)$ [log binned]')

In [ ]:
#### Theoretical degree distribution

def theo_pk(row):
    k = row['degree']
    prob_k = 12/(k*(k+1)*(k+2))
    return prob_k

dt_df2['theo_pk'] = dt_df2.apply(theo_pk, axis = 1)
dt_df2

In [ ]:
lb_theo_pk = logbin_px(dt_df2, 'theo_pk')
lb_theo_pk

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_theo_pk['X_min'], lb_theo_pk['Y_prob'], color = 'r', label = 'theoretical $p(k)$')
ax.loglog(lb_deg2['X_min'], lb_deg2['Y_prob'], label = 'calculated')
ax.set_xlabel('degree, $k$ [log binned]')
ax.set_ylabel('$p(k)$ [log binned]')
ax.legend()

In [ ]:
lb_knn2 = logbin_fx2(dt_agg_df2, 'degree', 'knn(k)')
lb_clust2 = logbin_fx2(dt_agg_df2, 'degree', 'c(k)')

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_knn2['X_min'], lb_knn2['Y_mean'])
ax.set_xlabel('degree, $k$ [log binned]')
ax.set_ylabel('knn(k) [log binned, mean]')

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_clust2['X_min'], lb_clust2['Y_mean'])
ax.set_xlabel('degree, $k$ [log binned]')
ax.set_ylabel('c(k) [log binned, mean]')


##### Explanation

* Degree distribution: 
Exhibits power-law distribution - straight line in log-log plot. The theoretical $p(k)$ represents the 

* Degree-dependent clustering: 
It also exhibits a power-law distribution

* Average nearest neighbor degrees:



#### Problem 4: Extra Credit


In [ ]:
# Create degree, triangle count
degrees = {node:val for (node, val) in G.degree()}
triangles = nx.triangles(G)

# Create degree-dependent clustering values
def locclust(row):
    val = row['tri']/comb(row['degree'], 2)
    return val

# Create knn values
knn_val = {}
for main_node in G.nodes():
    knn = 0
    for neigh in G.neighbors(main_node):
        knn += degrees[neigh]
    knn_val[main_node] = knn/degrees[main_node]


# Create dataframe with 4 values
dt_df3 = pd.DataFrame.from_dict(degrees, orient = 'index', columns = ['degree'])
dt_df3['tri'] = triangles.values()
dt_df3['loc_clust'] = dt_df3.apply(locclust, axis = 1)
dt_df3['knn_val'] = knn_val.values()
dt_df3 = dt_df3.sort_values(by=['degree'])


dt_agg_df3 = dt_df3[['degree', 'loc_clust', 'knn_val']].groupby(['degree']).agg({'loc_clust':['mean'], 'knn_val':['mean']}).droplevel(axis=1, level=0).reset_index()
dt_agg_df3.columns = ['degree', 'c(k)', 'knn(k)']
dt_agg_df3.head()


In [ ]:
lb_clust3 = logbin_fx2(dt_agg_df3, 'degree', 'c(k)')
lb_clust3.head()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.loglog(lb_clust2['X_min'], lb_clust2['Y_mean'], label = 'simulated; n = 10e4')
ax.loglog(lb_clust3['X_min'], lb_clust3['Y_mean'], color = 'r', label = 'simulated; n = 10e2')
ax.set_xlabel('degree, $k$ [log binned]')
ax.set_ylabel('c(k) [log binned, mean]')
ax.legend()
plt.title('Degree dependent clustering comparison')